# IMPORTS

In [1]:
import numpy as np
from tensorflow.examples.tutorials.mnist import input_data
import tensorflow as tf
import matplotlib.pyplot as plt
import numpy as np

# MODEL IMPLEMENTATION

In [19]:
#GLOBAL PARAMS

LR = 0.001
EPOCHS = 1500
BATCH_SIZE = 128

HEIGHT=28
WIDTH=28
INPUT_NUM = HEIGHT*WIDTH

CHANNELS_NUM=1
CLASS_NUM=10

In [20]:
def CONV_NN(X, reuse, is_training):
    
    with tf.variable_scope('Conv_nn', reuse=reuse):
        X = tf.reshape(X, shape=[-1, HEIGHT, WIDTH, CHANNELS_NUM])
        lay1 = tf.layers.max_pooling2d(tf.layers.conv2d(X, 16, 4, activation=tf.nn.relu), 2, 2)
        lay2 = tf.layers.max_pooling2d(tf.layers.conv2d(lay1, 32, 2, activation=tf.nn.relu), 2, 2)
        lay3 = tf.layers.dense(tf.contrib.layers.flatten(lay2), 512)
        lay4 = tf.layers.dropout(lay3, rate=0.2, training=is_training)
        lay5 = tf.layers.dense(lay4, CLASS_NUM)
        
        return lay5

def model_fn(features, labels, mode):
    
    X = features[FEAT_NAME]
    
    #train logits
    #reuse is set to false, because layers' variables are non-existent, thus created not reused
    l_train = CONV_NN(X, False, True)
    #test logits (reusing trained variables, is_training is set to false, so there's no dropout)
    l_test = CONV_NN(X, True, False)
    
    if mode == tf.estimator.ModeKeys.PREDICT:
        #just using trained layers and aplying arg_max function so the class with highest probability is chosen
        return tf.estimator.EstimatorSpec(mode, predictions=tf.argmax(l_test, axis=1))
    
    #optimizer
    optimizer = tf.train.AdamOptimizer(learning_rate=LR)
    #loss function for logits
    loss_func = tf.reduce_mean(tf.nn.sparse_softmax_cross_entropy_with_logits(logits=l_train, 
                                                                             labels=tf.cast(labels, dtype=tf.int32)))
    
    #construction of training (loss function optimization) process
    training_func = optimizer.minimize(loss_func, global_step=tf.train.get_global_step())
    #contruction of accuracy calculation
    accuracy_func = tf.metrics.accuracy(labels=labels, predictions=tf.argmax(l_test, axis=1))
    
    #the model_fn function must return EstimatorSpec object
    
    return tf.estimator.EstimatorSpec(mode=mode, 
                                     predictions=tf.argmax(l_test, axis=1), 
                                     loss=loss_func, 
                                     train_op=training_func, 
                                     eval_metric_ops={'accuracy': accuracy_func})
    
model = tf.estimator.Estimator(model_fn)

FEAT_NAME = 'images'
mnist = input_data.read_data_sets("/tmp/data/", one_hot=False)

INFO:tensorflow:Using default config.
INFO:tensorflow:Using config: {'_model_dir': 'C:\\Users\\micha\\AppData\\Local\\Temp\\tmp4rm850c0', '_tf_random_seed': None, '_save_summary_steps': 100, '_save_checkpoints_steps': None, '_save_checkpoints_secs': 600, '_session_config': allow_soft_placement: true
graph_options {
  rewrite_options {
    meta_optimizer_iterations: ONE
  }
}
, '_keep_checkpoint_max': 5, '_keep_checkpoint_every_n_hours': 10000, '_log_step_count_steps': 100, '_train_distribute': None, '_device_fn': None, '_protocol': None, '_eval_distribute': None, '_experimental_distribute': None, '_experimental_max_worker_delay_secs': None, '_session_creation_timeout_secs': 7200, '_service': None, '_cluster_spec': <tensorflow.python.training.server_lib.ClusterSpec object at 0x000002488D332710>, '_task_type': 'worker', '_task_id': 0, '_global_id_in_cluster': 0, '_master': '', '_evaluation_master': '', '_is_chief': True, '_num_ps_replicas': 0, '_num_worker_replicas': 1}
Extracting /tmp/d

# DEFINING INPUT AND MODEL TRAINING

In [21]:
train_input_fn = tf.estimator.inputs.numpy_input_fn(
    x={FEAT_NAME:mnist.train.images},
    y=mnist.train.labels,
    batch_size=BATCH_SIZE,
    num_epochs=None, 
    shuffle=True
)

test_input_fn = tf.estimator.inputs.numpy_input_fn(
    x={FEAT_NAME: mnist.test.images}, y=mnist.test.labels,
    batch_size=BATCH_SIZE, shuffle=False)

In [22]:
model.train(train_input_fn, steps=EPOCHS)

INFO:tensorflow:Calling model_fn.
INFO:tensorflow:Done calling model_fn.
INFO:tensorflow:Create CheckpointSaverHook.
INFO:tensorflow:Graph was finalized.
INFO:tensorflow:Running local_init_op.
INFO:tensorflow:Done running local_init_op.
INFO:tensorflow:Saving checkpoints for 0 into C:\Users\micha\AppData\Local\Temp\tmp4rm850c0\model.ckpt.
INFO:tensorflow:loss = 2.2989922, step = 1
INFO:tensorflow:global_step/sec: 26.6426
INFO:tensorflow:loss = 0.18258509, step = 101 (3.756 sec)
INFO:tensorflow:global_step/sec: 26.1478
INFO:tensorflow:loss = 0.17597947, step = 201 (3.822 sec)
INFO:tensorflow:global_step/sec: 26.5829
INFO:tensorflow:loss = 0.10506825, step = 301 (3.763 sec)
INFO:tensorflow:global_step/sec: 26.5646
INFO:tensorflow:loss = 0.03280796, step = 401 (3.764 sec)
INFO:tensorflow:global_step/sec: 27.0258
INFO:tensorflow:loss = 0.053014863, step = 501 (3.701 sec)
INFO:tensorflow:global_step/sec: 26.5951
INFO:tensorflow:loss = 0.031329326, step = 601 (3.758 sec)
INFO:tensorflow:glob

# MODEL EVALUATION

In [23]:
model.evaluate(test_input_fn)

INFO:tensorflow:Calling model_fn.
INFO:tensorflow:Done calling model_fn.
INFO:tensorflow:Starting evaluation at 2020-07-03T22:15:42Z
INFO:tensorflow:Graph was finalized.
INFO:tensorflow:Restoring parameters from C:\Users\micha\AppData\Local\Temp\tmp4rm850c0\model.ckpt-1500
INFO:tensorflow:Running local_init_op.
INFO:tensorflow:Done running local_init_op.
INFO:tensorflow:Finished evaluation at 2020-07-03-22:15:43
INFO:tensorflow:Saving dict for global step 1500: accuracy = 0.9854, global_step = 1500, loss = 0.04327173
INFO:tensorflow:Saving 'checkpoint_path' summary for global step 1500: C:\Users\micha\AppData\Local\Temp\tmp4rm850c0\model.ckpt-1500


{'accuracy': 0.9854, 'loss': 0.04327173, 'global_step': 1500}